In [2]:
from dotenv import load_dotenv
from utils.llm import LLMWorker
from utils.config_loader import config

load_dotenv()
llm = LLMWorker(config).llm

In [6]:
from typing import Dict
from pydantic import BaseModel, Field
from langchain_core.tools import tool

@tool('calculate_tool')
def calculate_tool(a: int, b:int) -> int:
    """Используй чтобы сложить два числа"""
    return a + b

@tool('weather_tool')
def weather_tool() -> str:
    "Используй, чтобы узнать погоду"
    return "Сегодня солнечно!"

llm_with_tools  = llm.bind_tools([calculate_tool, weather_tool])

In [ ]:
llm_with_tools.invoke("Как дела?")

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.446122 seconds
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.790779 seconds


In [ ]:
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5714039062ae4282913bb2fe', 'function': {'arguments': '{"a":2,"b":100}', 'name': 'calculate_tool'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 222, 'total_tokens': 246, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1755704796-SOp1jEjIVVbbEqRSqlA6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3159d65e-5176-4556-8453-764387ff4d9c-0', tool_calls=[{'name': 'calculate_tool', 'args': {'a': 2, 'b': 100}, 'id': 'call_5714039062ae4282913bb2fe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 222, 'output_tokens': 24, 'total_tokens': 246, 'input_token_details': {}, 'output_token_details': {}})

In [3]:
from utils.models import AgentState
from utils.jira_api import get_tasks

state = AgentState({
            'input': None,
            'dialog': None,
            'names': None,
            'classify_result': None,
            'users': None,
            'accounts': None,
            'extracted_tasks': None,
            'summary': None,
            'feedback': None,
            'report': None,
            'jira_tasks': None,
            'matched_tasks': None
        })

state['jira_tasks'] = get_tasks()
state['jira_tasks']


[{'id': 'KAN-11',
  'name': 'Подготовка кейсов для презентации',
  'description': 'Олег должен подготовить кейсы для презентации модуля прогнозирования спроса',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-10',
  'name': 'Демо-версия модуля 1С',
  'description': 'Антон должен подготовить демо-версию модуля 1С к 15-му числу',
  'reporter': 'Максим Абрамов',
  'assigned': 'Антон'},
 {'id': 'KAN-9',
  'name': 'Тестирование сценариев оповещений',
  'description': 'Олег должен протестировать сценарии оповещений о критичных остатках после интеграции с 1С',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-8',
  'name': 'Изучение аналогичных решений',
  'description': 'Олег продолжает изучать аналогичные решения',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-7',
  'name': 'Оценка сроков на ТЗ для прогнозирования спроса',
  'description': 'Антон должен оценить сроки на ТЗ для модуля прогнозирования спроса',
  'reporter': 'Максим А

In [4]:
from utils.nodes import match_tasks_node

dummy_tasks = [
    {
        'name': 'Корректировка ТЗ по срокам',
        'description': 'Максим Абрамов скорректирует ТЗ для модуля прогнозирования спроса',
        'reporter':'Максим Абрамов',
        'assigned':'Максим Абрамов'
    },
    {
        'name': 'Валидирование интеграции 1C',
        'description': 'Исправить ускорение интеграции 1С, так как нашлись баги',
        'reporter':'Антон',
        'assigned':'Антон'
    }
]
state['extracted_tasks'] = dummy_tasks
out_state = match_tasks_node(state)
out_state['matched_tasks']

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


{'new_tasks': [],
 'unchanged_tasks': [{'id': 'KAN-11',
   'name': 'Подготовка кейсов для презентации',
   'description': 'Олег должен подготовить кейсы для презентации модуля прогнозирования спроса',
   'reporter': 'Максим Абрамов',
   'assigned': 'Олег'},
  {'id': 'KAN-10',
   'name': 'Демо-версия модуля 1С',
   'description': 'Антон должен подготовить демо-версию модуля 1С к 15-му числу',
   'reporter': 'Максим Абрамов',
   'assigned': 'Антон'},
  {'id': 'KAN-9',
   'name': 'Тестирование сценариев оповещений',
   'description': 'Олег должен протестировать сценарии оповещений о критичных остатках после интеграции с 1С',
   'reporter': 'Максим Абрамов',
   'assigned': 'Олег'},
  {'id': 'KAN-8',
   'name': 'Изучение аналогичных решений',
   'description': 'Олег продолжает изучать аналогичные решения',
   'reporter': 'Максим Абрамов',
   'assigned': 'Олег'},
  {'id': 'KAN-6',
   'name': 'Прогнозирование спроса',
   'description': 'Запустить модуль прогнозирования спроса до конца квартал

In [8]:
import json

print(json.dumps(out_state['matched_tasks'], indent=4, ensure_ascii=False))

{
    "new_tasks": [],
    "unchanged_tasks": [
        {
            "id": "KAN-11",
            "name": "Подготовка кейсов для презентации",
            "description": "Олег должен подготовить кейсы для презентации модуля прогнозирования спроса",
            "reporter": "Максим Абрамов",
            "assigned": "Олег"
        },
        {
            "id": "KAN-10",
            "name": "Демо-версия модуля 1С",
            "description": "Антон должен подготовить демо-версию модуля 1С к 15-му числу",
            "reporter": "Максим Абрамов",
            "assigned": "Антон"
        },
        {
            "id": "KAN-9",
            "name": "Тестирование сценариев оповещений",
            "description": "Олег должен протестировать сценарии оповещений о критичных остатках после интеграции с 1С",
            "reporter": "Максим Абрамов",
            "assigned": "Олег"
        },
        {
            "id": "KAN-8",
            "name": "Изучение аналогичных решений",
            "descript

In [4]:
import json

matched_tasks = json.loads(out_state['matched_tasks'])
updated_tasks = matched_tasks['updated_tasks']
new_tasks = matched_tasks['new_tasks']
unchanged_tasks = matched_tasks['unchanged_tasks']

print('updated tasks')
for item in updated_tasks:
    print(json.dumps(item, indent=4, ensure_ascii=False))

updated tasks
{
    "id": "10069",
    "name": "Оценка сроков на ТЗ для прогнозирования спроса",
    "description": "Антон должен оценить сроки на ТЗ для модуля прогнозирования спроса | Максим Абрамов скорректирует ТЗ для модуля прогнозирования спроса",
    "reporter": "Максим Абрамов",
    "assigned": "Антон"
}
{
    "id": "10066",
    "name": "Интеграция с 1С",
    "description": "Ускорить интеграцию с 1С, так как это критично для клиента | Исправить ускорение интеграции 1С, так как нашлись баги",
    "reporter": "Максим Абрамов",
    "assigned": "Антон"
}


In [5]:
print('new tasks')
for item in new_tasks:
    print(json.dumps(item, indent=4, ensure_ascii=False))

new tasks


In [6]:
print('unchanged tasks')
for item in unchanged_tasks:
    print(json.dumps(item, indent=4, ensure_ascii=False))

unchanged tasks
{
    "id": "10074",
    "name": "Подготовка кейсов для презентации",
    "description": "Олег должен подготовить кейсы для презентации модуля прогнозирования спроса",
    "reporter": "Максим Абрамов",
    "assigned": "Олег"
}
{
    "id": "10073",
    "name": "Демо-версия модуля 1С",
    "description": "Антон должен подготовить демо-версию модуля 1С к 15-му числу",
    "reporter": "Максим Абрамов",
    "assigned": "Антон"
}
{
    "id": "10071",
    "name": "Тестирование сценариев оповещений",
    "description": "Олег должен протестировать сценарии оповещений о критичных остатках после интеграции с 1С",
    "reporter": "Максим Абрамов",
    "assigned": "Олег"
}
{
    "id": "10070",
    "name": "Изучение аналогичных решений",
    "description": "Олег должен изучить аналогичные решения для модуля прогнозирования спроса",
    "reporter": "Максим Абрамов",
    "assigned": "Олег"
}
{
    "id": "10068",
    "name": "Прогнозирование спроса",
    "description": "Запустить модуль

In [2]:
from utils.jira_api import update_task

print(update_task(task_id='KAN-8', description='Олег продолжает изучать аналогичные решения'))

Status code:  204

{}
Status code:  204

{}


In [1]:
from utils import jira_api

d = jira_api.get_tasks()
d

Status code:  400
{"errorMessages":[],"errors":{"description":"Operation value must be an Atlassian Document (see the Atlassian Document Format)"}}
{}


[{'id': 'KAN-11',
  'name': 'Подготовка кейсов для презентации',
  'description': 'Олег должен подготовить кейсы для презентации модуля прогнозирования спроса',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-10',
  'name': 'Демо-версия модуля 1С',
  'description': 'Антон должен подготовить демо-версию модуля 1С к 15-му числу',
  'reporter': 'Максим Абрамов',
  'assigned': 'Антон'},
 {'id': 'KAN-9',
  'name': 'Тестирование сценариев оповещений',
  'description': 'Олег должен протестировать сценарии оповещений о критичных остатках после интеграции с 1С',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-8',
  'name': 'Изучение аналогичных решений',
  'description': 'Олег должен изучить аналогичные решения для модуля прогнозирования спроса',
  'reporter': 'Максим Абрамов',
  'assigned': 'Олег'},
 {'id': 'KAN-7',
  'name': 'Оценка сроков на ТЗ для прогнозирования спроса',
  'description': 'Антон должен оценить сроки на ТЗ для модуля прогнозирования сп

In [2]:
d[0].keys()

dict_keys(['id', 'name', 'description', 'reporter', 'assigned'])

In [3]:
d[0]['id']

'10074'

In [ ]:
d[0]['fields']

{'summary': 'Подготовка кейсов для презентации',
 'description': {'type': 'doc',
  'version': 1,
  'content': [{'type': 'paragraph',
    'content': [{'type': 'text',
      'text': 'Олег должен подготовить кейсы для презентации модуля прогнозирования спроса'}]}]},
 'reporter': {'self': 'https://dolor.atlassian.net/rest/api/3/user?accountId=62010cc3ff9289006eeaf4c6',
  'accountId': '62010cc3ff9289006eeaf4c6',
  'emailAddress': 'abramovmaxim.894@gmail.com',
  'avatarUrls': {'48x48': 'https://secure.gravatar.com/avatar/40818c0ff5a3794cac4e4a48a5bbae1d?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Fdefault-avatar-6.png',
   '24x24': 'https://secure.gravatar.com/avatar/40818c0ff5a3794cac4e4a48a5bbae1d?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Fdefault-avatar-6.png',
   '16x16': 'https://secure.gravatar.com/avatar/40818c0ff5a3794cac4e4a48a5bbae1d?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%

: 